<a href="https://colab.research.google.com/github/wangbxj1234/voxel/blob/main/voxel_hash_FNV64_1A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

x = torch.rand(100,6)####n*c
p = torch.rand(100,3) #n*3

p=p.detach()
x=x.detach()
discrete_coord = torch.floor(p *10)
discrete_coord[1]=discrete_coord[0]
discrete_coord[3]=discrete_coord[0]

In [2]:
def fnv_hash_vec(arr):
    """
    FNV64-1A
    """
    assert arr.ndim == 2
    # Floor first for negative coordinates
    arr = arr.copy()
    arr = arr.astype(np.uint64, copy=False)
    hashed_arr = np.uint64(14695981039346656037) * np.ones(arr.shape[0], dtype=np.uint64)
    for j in range(arr.shape[1]):
        hashed_arr *= np.uint64(1099511628211)
        hashed_arr = np.bitwise_xor(hashed_arr, arr[:, j])
    return hashed_arr

discrete_coord=discrete_coord.numpy()
key = fnv_hash_vec(discrete_coord)
key = key.astype(np.int16, copy=False)
key = torch.from_numpy(key)
key

tensor([ -1206,  -1206,   4879,  -1206,  -9022, -21172,   5734,  19591,    100,
        -27259, -12940,   1407, -27696,  17044, -22481,  -9462, -20309,   6179,
          9221,   4433,  -7718, -27695, -27250,  10475,  11776,  13076,  -5116,
         17043,   6180, -29868,  -7713,  -1213,  -9459,  -8590,  19164, -17705,
         20035,  25241,  26983, -31173, -16845, -27693,  12214, -29871, -10323,
        -16843,  19165, -15980,  26982, -19009,  -1208, -18584,  20039,   6620,
        -19015, -17268,   4447,  11344, -16848, -23345,   1405,   5736,  10040,
         13120,  17045,  -9457, -32477, -22916,  -9017, -23789,  13085, -17706,
          8790, -29866, -22481,  -4684, -30293,  25674,  12648,  -1635,  20466,
        -19877,  11783,  24372,  -9022,  16116,   9216, -21178,   4010, -22051,
        -31175, -19450, -17704,   4007,  24371,  12692,  30028, -16844,   -774,
         -9462], dtype=torch.int16)

In [3]:
m=key[:, None] - key[None,:]
a=(m == 0).nonzero() #find all the zeros ；（(m==0) is a boolean mask）

In [4]:
counts=torch.count_nonzero(m==0, dim=0)#count zeros row by row
minvoxel=torch.max(counts)#find the count of points in the smallest voxel grid.
minvoxel

tensor(3)

In [ ]:
###我希望 minvoxel 不要低于16， 但是我又不能把格子打的太粗糙。